# Token Classification (NER) with BERT

## Import the needed libraries

In [1]:
import json

import torch
print("Torch:", torch.__version__)
print("CUDA build:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())


Torch: 2.5.1+cu121
CUDA build: 12.1
CUDA available: True


In [65]:
import numpy as np
import torch
import tokenizers
import seqeval
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, BertTokenizerFast, pipeline,DataCollatorForTokenClassification

from datasets import load_dataset, Dataset, DatasetDict
from evaluate import load

## Load the dataset and analyze

In [3]:
conds=load_dataset("conll2003",trust_remote_code=True)
conds


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'conll2003' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
Using the latest cached version of the dataset since conll2003 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'conll2003' at C:\Users\gundo\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98 (last modified on Sun Sep  7 09:06:46 2025).


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
conds.shape

{'train': (14041, 5), 'validation': (3250, 5), 'test': (3453, 5)}

In [5]:
conds['train']

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [6]:
conds['train'].features['ner_tags']

List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']))

In [7]:
conds['train'].features['tokens']


List(Value('string'))

In [8]:
conds['train'].features['pos_tags']


List(ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']))

In [9]:
conds['train'].description


'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

In [10]:
tokenizer= BertTokenizerFast.from_pretrained('bert-base-uncased')

In [11]:
example_text =conds['train'][0]
example_text

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [12]:
tokenized_input= tokenizer(example_text['tokens'],is_split_into_words=True)
tokenized_input

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokens=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
word_ids = tokenized_input.word_ids()
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [14]:
tokens

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [15]:
len(example_text['ner_tags'])

9

In [16]:
len(tokenized_input['input_ids'])

11

In [17]:
#We need to align tokens and labels to get better results and make no room for inconsistency


In [18]:
def tokenize_and_align_labels(example,label_all_tokens=True):
    tokenized_input = tokenizer(example['tokens'],truncation=True,is_split_into_words=True)
    labels =[]

    for i,label in enumerate(example['ner_tags']):
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx=None
        label_ids=[]

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx!= previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_input['labels']=labels
    return tokenized_input






In [19]:
conds['train'][4:5]['ner_tags']

[[5,
  0,
  0,
  0,
  0,
  3,
  4,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [20]:
q = tokenize_and_align_labels(conds['train'][4:5])
print(q)

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]]}


In [21]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]),q['labels'][0]):
    print(f"{token:_<40} {label}")


[CLS]___________________________________ -100
germany_________________________________ 5
'_______________________________________ 0
s_______________________________________ 0
representative__________________________ 0
to______________________________________ 0
the_____________________________________ 0
european________________________________ 3
union___________________________________ 4
'_______________________________________ 0
s_______________________________________ 0
veterinary______________________________ 0
committee_______________________________ 0
werner__________________________________ 1
z_______________________________________ 2
##wing__________________________________ 2
##mann__________________________________ 2
said____________________________________ 0
on______________________________________ 0
wednesday_______________________________ 0
consumers_______________________________ 0
should__________________________________ 0
buy_____________________________________ 0
sheep___

In [22]:
#if it is 0 it is not recognized by the NER tags and do not have any tags

In [23]:
#fonskiyonumuzu main datasete uyguluyoruz
tokenized_dataset = conds.map(tokenize_and_align_labels,batched=True)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [24]:
tokenized_dataset['train']

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 14041
})

In [25]:
model=AutoModelForTokenClassification.from_pretrained('bert-base-uncased',num_labels=9)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from transformers import TrainingArguments,Trainer ,DataCollatorForTokenClassification


In [31]:
args=TrainingArguments('test-ner',eval_strategy='epoch',learning_rate=2e-5,per_device_train_batch_size=16,per_device_eval_batch_size=16,num_train_epochs=1,weight_decay=0.01)

In [32]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [38]:
metric = load('seqeval')

In [39]:
example=conds['train'][0]

In [42]:
label_list = conds['train'].features['ner_tags'].feature.names

In [43]:
label_list


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [44]:
labels = [label_list[i] for i in example['ner_tags']]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [47]:
metric.compute(predictions=[labels],references=[labels])

{'MISC': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(2)},
 'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(1)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(1.0),
 'overall_f1': np.float64(1.0),
 'overall_accuracy': 1.0}

In [54]:
# 1) Değerlendirme metriği
# 2) label_list zaten var:
# label_list = conds['train'].features['ner_tags'].feature.names

# 3) compute_metrics: seqeval "sequence of sequences" ister
def compute_metrics(eval_preds):
    logits, labels = eval_preds                      # (batch, seq_len, num_labels), (batch, seq_len)
    preds = np.argmax(logits, axis=-1)               # (batch, seq_len)

    true_predictions = [
        [label_list[p] for p, l in zip(pred, lab) if l != -100]
        for pred, lab in zip(preds, labels)
    ]
    true_labels = [
        [label_list[l] for p, l in zip(pred, lab) if l != -100]
        for pred, lab in zip(preds, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall":    results["overall_recall"],
        "f1":        results["overall_f1"],
        "accuracy":  results["overall_accuracy"],
    }


In [56]:
trainer = Trainer(model=model,args=args,train_dataset=tokenized_dataset['train'],eval_dataset=tokenized_dataset['validation'],tokenizer=tokenizer,data_collator=data_collator,compute_metrics=compute_metrics)

C:\Users\gundo\AppData\Local\Temp\ipykernel_19476\2963141010.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,args=args,train_dataset=tokenized_dataset['train'],eval_dataset=tokenized_dataset['validation'],tokenizer=tokenizer,data_collator=data_collator,compute_metrics=compute_metrics)


In [57]:
trainer.train()

model.save_pretrained('ner_model')
tokenizer.save_pretrained('tokenizer')

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.217700,0.069589,0.904511,0.921915,0.913130,0.980809


('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [58]:
id2label = {str(i):label for i,label in  enumerate(label_list)}

label2id = {
    label: str(i)  for i,label in enumerate(label_list)
}

In [60]:
import json
config= json.load(open('ner_model/config.json'))

In [62]:
config['id2label']= id2label
config['label2id']==label2id
json.dump(config,open('ner_model/config.json','w'))

model_fine_tuned= AutoModelForTokenClassification.from_pretrained('ner_model')

In [75]:
nlp = pipeline('ner',model=model_fine_tuned,tokenizer=tokenizer)
text="Bill Gates is the Founder of Microsoft."
ner_results=nlp(text)
print(ner_results)

Device set to use cuda:0


[{'entity': 'B-PER', 'score': np.float32(0.9666577), 'index': 1, 'word': 'bill', 'start': 0, 'end': 4}, {'entity': 'I-PER', 'score': np.float32(0.9428921), 'index': 2, 'word': 'gates', 'start': 5, 'end': 10}, {'entity': 'B-ORG', 'score': np.float32(0.9608258), 'index': 7, 'word': 'microsoft', 'start': 29, 'end': 38}]
